* All crime codes can be divided into multiple categories to facilitate text generation:
    * crime type- bias, hate, assault, vandalism etc.
    * victim description
    * suspect description
    * scene description
    * victim behavior details<br>
This can be learned using code description and code through clustering of codes based on description similarity.Further Data Collection will enable neural network implementation of this as well as full sentence formation using words for these codes(https://openreview.net/pdf?id=Skvgqgqxe).
    
* The **general template** of crime paragraph is of the form: <br>

    * ASSAULT(OTHER ASSAULT/ BATTERY): The suspect, <suspect description> 'crime type from Crm Cd Desc'(ed) <victim description using fields on victim details like gender, age, disabled etc.> near <crime location description collected> while <comments from victim behavior codes like 0359>. <br> 
    Ex -**The suspect** assaulted **the 59-year-old black disabled male victim** near **the Wilshire/Vermont metro station**, while **shouting an "unusual" statements**.<br>
    * VANDALISM(Misdemeanor): The suspect vandalised < victim description > resulting in damage worth < damage amount >
    * Similar templates for other forms of crime 
    
* If the field 'Crm Cd Desc' is not available, the fallback is the most common uni-gram in the first part of all paragraphs for that particular crime description code.Similar approach for other codes.The first part of sentence is decided by considering length that's twice of postion of first VERB token in the sentence.<br> 
    
* Depending on the crime type, additional sentences can be generated based on additional informattion collected. For example, a field indicating whether or not the victim knew the suspect is generally collected for 'hate crime'. So a sentence like "The suspect <didn't/ know> the victim from before" can be added to the paragraph.<br>

* A manual check of this template over 25 samples shows it's exactly matching(in meaning) with the original paragraph, 3 were also correct in crime communication but also had additional information about both victim and suspect.- This can be taken care of when training model for sentence generation with large training data. Till then, manual verification is needed.

In [2]:
import pandas as pd

In [4]:
merged_data = pd.read_csv('../data/merged_records_df.csv')
merged_data.columns


Index(['Crime', 'Datetime', 'Description', 'Location_x', 'Neighborhood_x',
       'Suspect Actions', 'Weapon', 'Number', '@Photos', 'norm_date', 'Dr No',
       'Area ID', 'Area Name', 'Crm Cd', 'Crm Cd 1', 'Crm Cd 2', 'Crm Cd 3',
       'Crm Cd 4', 'Crm Cd Desc', 'Cross Street', 'Date Occ', 'Date Rp Td',
       'Latitude', 'Location_y', 'Location 1', 'Longitude', 'Mo Codes',
       'Neighborhood_y', 'Neighborhood Council', 'Prem Is Cd', 'Prem Is Desc',
       'Rpt Dist No', 'Status', 'Status Desc', 'Time Occ', 'Tract', 'Vict Age',
       'Vict Descent', 'Vict Sex', 'Weapon Desc', 'Weapon Used Cd',
       'Zip Code'],
      dtype='object')

In [5]:
merged_data[['Description', 'Mo Codes', 'Crm Cd', 'Crm Cd 1', 'Crm Cd 2', 'Crm Cd 3',
       'Crm Cd 4', 'Crm Cd Desc', 'Prem Is Cd', 'Weapon Used Cd']]

,Description,Mo Codes,Crm Cd,Crm Cd 1,Crm Cd 2,Crm Cd 3,Crm Cd 4,Crm Cd Desc,Prem Is Cd,Weapon Used Cd
0,The suspect vandalized property at Marmion Apa...,0903 1536 1539 2054,745.0,745.0,NaN,NaN,NaN,VANDALISM - MISDEAMEANOR ($399 OR UNDER),502.0,NaN
1,The suspect assaulted the 59-year-old black di...,0359 1238 2057 1506 0305 0429 0903,624.0,624.0,946.0,NaN,NaN,BATTERY - SIMPLE ASSAULT,102.0,400.0
2,The suspect assaulted the 18-year-old Latinx f...,0919 0903 2056 1516,230.0,230.0,NaN,NaN,NaN,"ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT",122.0,307.0
3,The suspect vandalized the Carson-Gore Academy...,0329 0903 1414 1528 2053,745.0,745.0,946.0,NaN,NaN,VANDALISM - MISDEAMEANOR ($399 OR UNDER),704.0,NaN
4,The suspect spat on the 17-year-old white male...,1251 1259 0319 2035 2053 1528 2004 0903 0356,625.0,625.0,NaN,NaN,NaN,OTHER ASSAULT,102.0,500.0
5,The suspect hit the 27-year-old Hispanic male ...,0903 1218 2004 0416 0319 2054 1539,624.0,624.0,NaN,NaN,NaN,BATTERY - SIMPLE ASSAULT,102.0,308.0
6,The suspect attacked the 42-year-old transgend...,0416 0446 0913 1509 1510 1536 0903 0910 2006 0906,624.0,624.0,NaN,NaN,NaN,BATTERY - SIMPLE ASSAULT,124.0,400.0
7,The suspect attacked the 42-year-old transgend...,0444 0913 1510 0903 0906 1539 2054,624.0,624.0,NaN,NaN,NaN,BATTERY - SIMPLE ASSAULT,102.0,400.0
8,The suspect entered the Holy Martyrs Armenian ...,0903 1402 2055 1519,888.0,888.0,998.0,NaN,NaN,TRESPASSING,721.0,NaN
9,The suspect vandalized the nearby Temple Judea...,0329 0903 2053 1528,745.0,745.0,NaN,NaN,NaN,VANDALISM - MISDEAMEANOR ($399 OR UNDER),730.0,NaN


In [6]:
merged_data.iloc[0]['Description']

'The suspect vandalized property at Marmion Apartments, resulting in damage of $400 or less.'

In [8]:
merged_data.iloc[0]['Mo Codes']

'0903 1536 1539 2054'

In [9]:
merged_data.iloc[1]['Description']

'The suspect assaulted the 59-year-old black disabled male victim near the Wilshire/Vermont metro station, while shouting an "unusual" statement at him. The suspect did not know the victim prior to the crime.'

In [11]:
merged_data.iloc[1]['Mo Codes']

'0359 1238 2057 1506 0305 0429 0903'